# Covid-19 Vaccine Allocations and Case Counts Analysis
----

Written in the Python 3.7.9 Environment

By Tarak Patel, Nicole Lund and Anne Niemiec.

Common Jupyter cells have been notated for ease of merging after individuals their analysis.

Individual working files will be merged using https://pypi.org/project/nbmerge/

In [1]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load Dependencies

# Data Organization & Collection
import pandas as pd
import requests
import json
from census import Census

# Data Visualization & Analysis
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import gmaps

# General Navigation
import time
import os
import sys

In [2]:
# Define users api key file folder location
api_key_path = r"C:\Users\nlund\Documents\GitHub\untracked_files"

In [3]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Add API key file to system path
sys.path.append(api_key_path)

# Import Census API Key
from untracked_api_keys import census_key
c = Census(census_key, year=2019)

# Import Google API key
from untracked_api_keys import g_key

ModuleNotFoundError: No module named 'untracked_api_keys'

In [4]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load clean data for Case and Death Counts by state and time
    # This data combines:
        # United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv'
        # state_code.csv

c_d_withstate_df_path = r"..\data_manipulation\clean_data\c_d_withstate_df.csv"
cases_deaths_df = pd.read_csv(c_d_withstate_df_path)
cases_deaths_df

,state_name,state_code,date,day_num,total_cases,confirmed_cases,probable_cases,new_cases,pnew_cases,total_deaths,confirmed_deaths,probable_deaths,new_deaths,pnew_deaths,consent_cases,consent_deaths,created_at
0,Alaska,AK,12/14/2020,0,40160,0.0,0.0,422,0.0,175,0.0,0.0,0,0.0,0,0,12/15/2020 14:39
1,Alabama,AL,12/14/2020,0,297895,244762.0,53133.0,2264,332.0,4102,3624.0,478.0,0,0.0,Agree,Agree,12/15/2020 14:39
2,Arkansas,AR,12/14/2020,0,187057,0.0,0.0,1355,122.0,2990,0.0,0.0,45,22.0,Not agree,Not agree,12/15/2020 14:39
3,American Samoa,AS,12/14/2020,0,3,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,12/15/2020 14:39
4,Arizona,AZ,12/14/2020,0,420248,404294.0,15954.0,11806,503.0,7358,6782.0,576.0,1,-2.0,Agree,Agree,12/15/2020 14:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,Vermont,VT,4/17/2021,124,21869,0.0,0.0,66,2.0,242,0.0,0.0,0,0.0,Not agree,Not agree,4/18/2021 13:41
7496,Washington,WA,4/17/2021,124,385549,0.0,0.0,1655,185.0,5380,0.0,0.0,0,0.0,0,0,4/18/2021 13:41
7497,Wisconsin,WI,4/17/2021,124,650949,589940.0,61009.0,868,141.0,7422,6711.0,711.0,10,2.0,Agree,Agree,4/18/2021 13:41
7498,West Virginia,WV,4/17/2021,124,148517,117164.0,31353.0,446,97.0,2780,0.0,0.0,3,0.0,Agree,Not agree,4/18/2021 13:41


In [5]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load clean data for Vaccination Allocation Data by state and time
    # This data combines:
        # COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Pfizer.csv
        # COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Moderna.csv
        # COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Janssen.csv

Combined_Vaccine_df_path = r"..\data_manipulation\clean_data\Combined_Vaccine_df.csv"
vaccine_allocation_df = pd.read_csv(Combined_Vaccine_df_path)
vaccine_allocation_df.head()

,Unnamed: 0,Jurisdiction,Week of Allocations,PFE Dose-1,PFE Dose-2,MRNA Dose-1,MRNA Dose-2,JNJ Dose-1,Unnamed: 8
0,0,Connecticut,4/19/2021,54990,54990,40400,40400,0,NaN
1,1,Maine,4/19/2021,21060,21060,15400,15400,0,NaN
2,2,Massachusetts,4/19/2021,105300,105300,77700,77700,0,NaN
3,3,New Hampshire,4/19/2021,21060,21060,15500,15500,0,NaN
4,4,Rhode Island,4/19/2021,16380,16380,12100,12100,0,NaN


In [6]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load clean data for State Population from Census reports
    # This data collects census API request data by state:
        # Population.csv

Population_path = r"..\data_manipulation\clean_data\Population.csv"
census_df = pd.read_csv(Population_path)
census_df.head()

,state_name,state_code,population,median_age,household_income,per_capita_income,poverty_count,poverty_rate,unemployment_count,unemployment_rate
0,Alabama,AL,4876250.0,39.0,50536.0,27928.0,795989.0,16.323794,132095.0,2.708946
1,Alaska,AK,737068.0,34.3,77640.0,36787.0,76933.0,10.437707,26808.0,3.637114
2,Arizona,AZ,7050299.0,37.7,58945.0,30694.0,1043764.0,14.804535,195905.0,2.778676
3,Arkansas,AR,2999370.0,38.1,47597.0,26577.0,496260.0,16.545475,70481.0,2.349860
4,California,CA,39283497.0,36.5,75235.0,36955.0,5149742.0,13.109174,1199233.0,3.052765


# Group 1 Analysis

## Line Plot showing National Summarized Vaccine Allocations by Date
x-axis = Date (The data is reported weekly. To simplify the labels, convert the dates to a week number and then label by month.)

y-axis =

Line 1 - Pfizer + Moderna 1st dose allocation  
Line 2 - Pfizer + Moderna 2nd dose allocation  
Line 3 - J&J allocations  
Note: y-axis data is a summation for all states  
Legend labels = Pfizer + Moderna 1st dose, Pfizer + Moderna 2nd dose, J&J Single dose

In [16]:
vaccine_allocation_df['Month'] = pd.DatetimeIndex(vaccine_allocation_df['Week of Allocations']).month
vaccine_allocation_df['Year'] = pd.DatetimeIndex(vaccine_allocation_df['Week of Allocations']).year
vaccine_allocation_df['Dose-1'] = vaccine_allocation_df['PFE Dose-1'] + vaccine_allocation_df['MRNA Dose-1']
vaccine_allocation_df['Dose-2'] = vaccine_allocation_df['PFE Dose-2'] + vaccine_allocation_df['MRNA Dose-2']





,Unnamed: 0,Jurisdiction,Week of Allocations,PFE Dose-1,PFE Dose-2,MRNA Dose-1,MRNA Dose-2,JNJ Dose-1,Month,Year,Dose-1
0,0,Connecticut,04/19/2021,54990,54990,40400.0,40400.0,0.0,4,2021,95390.0
1,1,Maine,04/19/2021,21060,21060,15400.0,15400.0,0.0,4,2021,36460.0
2,2,Massachusetts,04/19/2021,105300,105300,77700.0,77700.0,0.0,4,2021,183000.0
3,3,New Hampshire,04/19/2021,21060,21060,15500.0,15500.0,0.0,4,2021,36560.0
4,4,Rhode Island,04/19/2021,16380,16380,12100.0,12100.0,0.0,4,2021,28480.0
...,...,...,...,...,...,...,...,...,...,...,...
1192,1192,Virginia,12/14/2020,72150,72150,0.0,0.0,0.0,12,2020,72150.0
1193,1193,Washington,12/14/2020,62400,62400,0.0,0.0,0.0,12,2020,62400.0
1194,1194,West Virginia,12/14/2020,16575,16575,0.0,0.0,0.0,12,2020,16575.0
1195,1195,Wisconsin,12/14/2020,49725,49725,0.0,0.0,0.0,12,2020,49725.0


## Line Plot showing National Vaccine Allocations by Date
x-axis = Date (The data is reported weekly. To simplify the labels, convert the dates to a week number and then label by month.)

y-axis =

Line 1 - Pfizer 1st dose allocation  
Line 2 - Moderna 1st dose allocations  
Line 3 - Pfizer 2nd dose allocations  
Line 4 - Moderna 2nd dose allocations  
Line 5 - J&J allocations  
Note: y-axis data is a summation for all states  
Legend labels = Pfizer 1st dose, Moderna 1st dose, Pfizer 2nd dose, Moderna 2nd dose, J&J Single dose

## Line Plot showing Individual State (Arizona) Vaccine Allocations by Date  
x-axis = Date (The data is reported weekly. To simplify the labels, convert the dates to a week number and then label by month.)

y-axis =

Line 1 - Pfizer 1st dose allocation  
Line 2 - Moderna 1st dose allocations  
Line 3 - Pfizer 2nd dose allocations  
Line 4 - Moderna 2nd dose allocations  
Line 5 - J&J allocations  
Legend labels = Pfizer 1st dose, Moderna 1st dose, Pfizer 2nd dose, Moderna 2nd dose, J&J Single dose

## Bonus Line Plot showing Individual State (User Choice) Vaccine Allocations by Date
x-axis = Date (The data is reported weekly. To simplify the labels, convert the dates to a week number and then label by month.)

y-axis =

Line 1 - Pfizer 1st dose allocation  
Line 2 - Moderna 1st dose allocations  
Line 3 - Pfizer 2nd dose allocations  
Line 4 - Moderna 2nd dose allocations  
Line 5 - J&J allocations  
Legend labels = Pfizer 1st dose, Moderna 1st dose, Pfizer 2nd dose, Moderna 2nd dose, J&J Single dose